In [23]:
!pip  install pandas
!pip  install numpy
!pip  install datetime


In [25]:
# ============================================================
# COMPARACIÓN MULTIFUENTE: PARO DOCENTE 2023
# Webscraping (web) vs SGI vs BigQuery (bq)
# ============================================================

import pandas as pd
from datetime import timedelta
import numpy as np

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  CARGA DE DATOS
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pandas as pd

web = pd.read_csv("noticias_paros_docentes_2023_ANALIZADO (1).csv", encoding="utf-8", low_memory=False)
bq  = pd.read_csv("bq-results-20251023-000630-1761178021663.csv", encoding="utf-8", low_memory=False)
sgi = pd.read_excel("Base SGI suspensión clases 2012-2025 03_08_2025_VALIDACION SED.xlsx")


# ===============================
# 🧹 3. Normalizar nombres de columnas
# ===============================
for df in [web, bq, sgi]:
    df.columns = df.columns.str.lower().str.strip()

# Crear columna de fuente
web["fuente"] = "web"
bq["fuente"] = "bq"
sgi["fuente"] = "sgi"

# ===============================
# 🔗 4. Crear columna 'url_limpia' (para evitar duplicados)
# ===============================
for df in [web, bq, sgi]:
    if "url" in df.columns:
        df["url_limpia"] = df["url"].astype(str).str.split("&ved=").str[0].str.strip()
    else:
        df["url_limpia"] = None

# ===============================
# 🔍 5. Filtrar solo noticias relacionadas con paro docente
# ===============================
palabras_clave = "paro docente|fecode|maestro|profesor|docente|paros de docentes|huelga de maestros"

def filtrar_paro(df):
    texto = df.astype(str).apply(lambda x: " ".join(x), axis=1).str.lower()
    return df[texto.str.contains(palabras_clave, na=False)]

web_fil = filtrar_paro(web)
bq_fil  = filtrar_paro(bq)
sgi_fil = filtrar_paro(sgi)

# ===============================
# 🚫 6. Quitar duplicados (por URL limpia)
# ===============================
urls_existentes = set(web_fil["url_limpia"].dropna())
bq_nuevas  = bq_fil[~bq_fil["url_limpia"].isin(urls_existentes)]
sgi_nuevas = sgi_fil[~sgi_fil["url_limpia"].isin(urls_existentes)]

# ===============================
# 🔀 7. Unir todo en una sola base
# ===============================
noticias_consolidada = pd.concat([web_fil, bq_nuevas, sgi_nuevas], ignore_index=True)

# ===============================
# 💾 8. Exportar
# ===============================
noticias_consolidada.to_csv("noticias_paro_docente_consolidada.csv", index=False, encoding="utf-8-sig")

print("✅ Base consolidada creada correctamente")
print(f"Total de noticias: {len(noticias_consolidada)}")
print(f"Nuevas de BQ: {len(bq_nuevas)} | Nuevas de SGI: {len(sgi_nuevas)}")
print(f"Archivo guardado en:\n{output_path}")

✅ Base consolidada creada correctamente
Total de noticias: 755
Nuevas de BQ: 532 | Nuevas de SGI: 66
Archivo guardado en:
C:/Users/angel/Documents/GitHub/SED---Cuantitativo/Webscrapping/Datos/noticias_paro_docente_consolidada.csv


In [26]:
# ============================================================
# 📂 1️⃣ CARGAR ARCHIVOS
# ============================================================
# Archivos: base consolidada y base de referencia (formato original)
consolidada_path = "noticias_paro_docente_consolidada.csv"
referencia_path  = "noticias_paros_docentes_2023_ANALIZADO (1).csv"

# Leer archivos
df = pd.read_csv(consolidada_path, encoding="utf-8", low_memory=False)
ref = pd.read_csv(referencia_path, encoding="utf-8", low_memory=False)

print(f"✅ Base consolidada: {df.shape[0]} filas")
print(f"✅ Base de referencia: {ref.shape[0]} filas")

# ============================================================
# 🧹 2️⃣ FILTRAR SOLO NOTICIAS DEL 2023
# ============================================================
# Crear columna combinada de texto para buscar "2023"
df["texto_completo"] = df.astype(str).apply(lambda x: " ".join(x), axis=1).str.lower()

# Filtrar solo aquellas que mencionan "2023"
df_2023 = df[df["texto_completo"].str.contains("2023", na=False)].copy()

print(f"🗓️ Noticias con año 2023: {len(df_2023)}")

# ============================================================
# 🚫 3️⃣ ELIMINAR DUPLICADOS (por URL o título)
# ============================================================
# Crear columna de URL limpia (eliminar parámetros de Google)
if "url" in df_2023.columns:
    df_2023["url_limpia"] = df_2023["url"].astype(str).str.split("&ved=").str[0].str.strip()
else:
    df_2023["url_limpia"] = None

# Eliminar duplicados por URL o título si existen
cols_dup = [c for c in ["url_limpia", "titulo", "title"] if c in df_2023.columns]
df_sin_dup = df_2023.drop_duplicates(subset=cols_dup, keep="first")

print(f"🧾 Noticias sin duplicados: {len(df_sin_dup)}")

# ============================================================
# 🧩 4️⃣ IGUALAR ESTRUCTURA AL ARCHIVO "ANALIZADO"
# ============================================================
# Usar las columnas del archivo de referencia
cols_ref = ref.columns
df_final = df_sin_dup.reindex(columns=cols_ref, fill_value=None)

print(f"📋 Columnas igualadas: {len(cols_ref)} columnas")

# ============================================================
# 💾 5️⃣ GUARDAR RESULTADO FINAL
# ============================================================
output_path = "noticias_paro_docente_2023_limpias.csv"
df_final.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ LIMPIEZA FINAL COMPLETADA")
print(f"Archivo guardado como: {output_path}")
print(f"Filas finales: {len(df_final)}")

✅ Base consolidada: 755 filas
✅ Base de referencia: 161 filas
🗓️ Noticias con año 2023: 178
🧾 Noticias sin duplicados: 123
📋 Columnas igualadas: 21 columnas
✅ LIMPIEZA FINAL COMPLETADA
Archivo guardado como: noticias_paro_docente_2023_limpias.csv
Filas finales: 123
